<a href="https://colab.research.google.com/github/shaky008/Genetic_Algorthim_Art/blob/main/FINAL_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image, ImageDraw
from random import randint
import matplotlib.pyplot as plt


# PATH
IMAGE_PATH = "homer.png"

# Generate random triangle points
def gen_random_triangle(width, height):
  points = np.array([
      [randint(0, width), randint(0, height)],
      [randint(0, width), randint(0, height)],
      [randint(0, width), randint(0, height)]
  ])
  return points


# Generate random color with transparency
def rand_color_with_alpha(prev_color=None):
  if prev_color is None:
      return np.append(np.random.randint(0, 256, 3), np.random.randint(50, 150))
  return np.clip(prev_color + np.random.randint(-20, 21, size=4), 0, 255)


# Function to calculate fitness score using RMSE
def cal_fitness(original, current):
  return np.sqrt(np.sum((original.astype(np.int32) - current.astype(np.int32)) ** 2))


# Mutate triangle by adjusting vertices randomly
def mutate_triangle(triangle, width, height):
  for i in range(3):
    if np.random.rand() < 0.5:
      triangle[i][0] = np.clip(triangle[i][0] + randint(-10, 10), 0, width) #vertix x
      triangle[i][1] = np.clip(triangle[i][1] + randint(-10, 10), 0, height)  #vertix y
  return triangle


# Upload image

original_image = Image.open(IMAGE_PATH).convert("RGBA").resize((640, 360))
original_array = np.array(original_image)
height, width, _ = original_array.shape
print(f"[INFO] Image resized to: {width} x {height}")

# Initialize blank image
best_image = np.ones_like(original_array) * 255
best_score = cal_fitness(original_array, best_image)

# Main optimization loop
max_no_generations = 38000
max_triangles = 350
cycle_rate = 10

triangles = []
colors = []
fitness_scores = []

for gen in range(1, max_no_generations + 1):
    temp_image = best_image.copy()
    best_local_score = np.inf
    best_local_triangle = None
    best_local_color = None

    for _ in range(cycle_rate):
        triangle = gen_random_triangle(width, height)
        color = rand_color_with_alpha()

        # Draw triangle on temp image
        img = Image.fromarray(temp_image.astype('uint8'))
        draw = ImageDraw.Draw(img, "RGBA")
        draw.polygon(list(map(tuple, triangle)), fill=tuple(color))
        temp_image_with_triangle = np.array(img)

        # Compute score
        score = cal_fitness(original_array, temp_image_with_triangle)
        if score < best_local_score:
            best_local_score = score
            best_local_triangle = triangle
            best_local_color = color

    # Update global best
    if best_local_score < best_score:
        best_score = best_local_score
        triangles.append(best_local_triangle)
        colors.append(best_local_color)

        img = Image.fromarray(best_image.astype('uint8'))
        draw = ImageDraw.Draw(img, "RGBA")
        draw.polygon(list(map(tuple, best_local_triangle)), fill=tuple(best_local_color))
        best_image = np.array(img)

    # Append fitness score for the current generation
    fitness_scores.append(best_score)

    # Visualize every 1000 generations
    if gen % 1000 == 0:
        for i in range(len(triangles)):
            mutated_triangle = mutate_triangle(triangles[i].copy(), width, height)

            # Draw the mutated triangle
            img = Image.fromarray(best_image.astype('uint8'))
            draw = ImageDraw.Draw(img, "RGBA")
            draw.polygon(list(map(tuple, mutated_triangle)), fill=tuple(colors[i]))
            temp_image_with_mutation = np.array(img)

            # Evaluate if the mutation improves the score
            mutated_score = cal_fitness(original_array, temp_image_with_mutation)
            if mutated_score < best_score:
                best_score = mutated_score
                triangles[i] = mutated_triangle  # Update the triangle in the list
                best_image = temp_image_with_mutation  # Update the best image

        plt.imshow(best_image)
        plt.title(f"Generation {gen}")
        plt.axis("off")
        plt.show()


# give final result
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(fitness_scores) + 1), fitness_scores, label="Fitness Score (RMSE)")
plt.title("Fitness Score Over Generations", fontsize=16)
plt.xlabel("Generation", fontsize=14)
plt.ylabel("Fitness Score (RMSE)", fontsize=14)
plt.grid(True)
plt.legend()
plt.show()




NameError: name 'IMAGE_PATH' is not defined